In [ ]:
import pickle

# Load Crop Recommendation Model
with open('crop_recommendation_model.pkl', 'rb') as file:
    crop_model = pickle.load(file)

# Load Fertilizer Prediction Model
with open('fertilizer_prediction_model.pkl', 'rb') as file:
    fertilizer_model = pickle.load(file)


In [ ]:
import pandas as pd

def recommend_crop_and_fertilizer(input_data):
    """
    Given input conditions, this function predicts the recommended crop
    and then suggests the best fertilizer for it.
    
    Parameters:
    input_data (dict): Contains features required for crop recommendation.
    
    Returns:
    dict: Contains the recommended crop and fertilizer.
    """

    # Convert input dictionary to DataFrame for model prediction
    input_df = pd.DataFrame([input_data])

    # Step 1: Predict the crop
    recommended_crop = crop_model.predict(input_df)[0]

    # Step 2: Prepare input for Fertilizer Model
    fertilizer_input = input_df.copy()
    fertilizer_input['Crop Type'] = recommended_crop  # Inject the predicted crop

    # Ensure that only relevant features for fertilizer prediction are used
    fertilizer_features = ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 
                           'Nitrogen', 'Potassium', 'Phosphorous']
    
    fertilizer_input = fertilizer_input[fertilizer_features]

    # Step 3: Predict the fertilizer
    recommended_fertilizer = fertilizer_model.predict(fertilizer_input)[0]

    return {
        "Recommended Crop": recommended_crop,
        "Recommended Fertilizer": recommended_fertilizer
    }

# Example input data
input_conditions = {
    "Temparature": 30,
    "Humidity": 60,
    "Moisture": 40,
    "Soil Type": "Loamy",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}

result = recommend_crop_and_fertilizer(input_conditions)
print(result)
